# Tidy data operations with R

So let's look at how to work with so-called tidy data. What exactly constitutes tidy data will be covered later. For now the dataset we use will be tidy.

In [ ]:
library(tidyverse)

Start by reading in a CSV file containing information about various nobel laureates.

The data set is mostly clean. For some people the exact birth date is not known so only the year is marked. We choose to omit this information.

Additionally some newer laureates are in the data set twice for some reason.

In [ ]:
nobel <- read_csv("nobel.csv", na=c("", "NA", "0000-00-00", "1898-00-00", "1943-00-00"))
nobel <- distinct(nobel, id, .keep_all=TRUE)

`read_csv()` returns a tibble, which is kind of like sugar frosting on top of a data frame. There are differences in default behaviour, but they are mostly not important for now.

The `head()` method displays the first few rows of the data set so you get an idea of what it contains.

In [ ]:
head(nobel)

`reac_csv()` was real smart, it recognized dates as such, numbers as such and mostly created facts as strings, not as factors.

In [ ]:
class(nobel$born)

In [ ]:
class(nobel$bornCountryCode)

## Filtering

The simplest tidy data operation that we can do is filter the dataset.

The syntax is the logical operator syntax we have learned earlier. You can chain multiple rules on the same variable or different variables.

In [ ]:
# laureates between 2008 (exclusive) and 2010 (inclusive)
filter(nobel, year =< 2010 & year > 2008))

In [ ]:
# female laureates born in Poland
filter(nobel, gender == "female" & bornCountryCode == "PL")

### Exercise

Try to filter female laureates who were born in either Poland or in Germany.

## Piping

The `%>%` operator is special syntax so we can easily chain operations.

For instance this 

```
arrange(
        filter(
            nobel, year < 2010 & year > 2008
        ), surname
    )
```
and this
```
intermediate <- filter(nobel, year < 2010 & year > 2008)
arrange(intermediate, surname)
```
are equivalent with this
```
filter(nobel, year < 2010 & year > 2008) %>%
    arrange(surname)
```

The `filter()` function expects the data as a first parameter but that is omitted in the piping syntax. The first parameter comes from the operation being piped from and the others are passed regularly.


This is the best thing since sliced bread, especially when the chain is very long, e.g. this one from later down the line.

```
filter (nobel, bornCountryCode != diedCountryCode) %>%
    group_by(bornCountryCode) %>%
    summarize(count = n()) %>%
    drop_na() %>%
    arrange(desc(count)) %>%
    head(20)
```

To prove that this works.


In [ ]:
filter(nobel, year < 2010 & year > 2008) %>%
    arrange(surname) %>%
    head(2)

## Selecting

Often we don't need all the data. To select some columns you can use the `select()` function.

In [ ]:
filter(nobel, year < 2010 & year > 2008) %>%
    arrange(surname) %>%
    select(firstname, surname, bornCountryCode)

In [ ]:
group_by(nobel, bornCountryCode) %>%
    summarize(count = n(), mean_fraction=mean(1/share)) %>%
    drop_na() %>%
    arrange(desc(count)) %>%
    head(20)

## Exercise

Do the same as above, except for countries where laureates died (and presumably are buried).

After death the fractions don't matter so that can be omitted.

Bonus, filter away people who have born and died in the same country.

This lets you get an idea of the winners (or losers) of brain drain.


## Exercise

Again, do the same analysis for the laureates, except only for the prizes on physics (arguably the only real science).

You can do the filter before or after the group_by, but best do it early.

Bonus, group by two variables, category and gender to get an understanding of how the awards in different fields are distributed. Now `head()` and `sort()` don't seem as logical as previously.

Which category has a third gender?

## Fun with `mutate()`

To compute aggregate statistics or combine things we have a function called `mutate()`.





In [ ]:
library(lubridate)


In [ ]:
summary(nobel$year)

In [ ]:
nobel2 <- filter(nobel, !is.na(year)) %>%
                mutate(
                    age = year - year(born)
                )
summary(nobel2$age)

In [ ]:
ggplot(nobel2, aes(x=category, y=age)) + geom_boxplot()


## Add ggplot

As it turns out you can just pass the results of operations to `ggplot`.

In [ ]:
# Compute the mean fraction of nobels awarded each year
data <- group_by(nobel2, year) %>%
    summarize(mean_fraction = mean(1/share)) 

# plot the raw data and plot a fitted curve
# what is the 
ggplot(data, aes(x=year, y=mean_fraction)) + geom_path() + geom_smooth()

# you could just combine the two using piping but the ggplot syntax makes that a bit iffy

ggplot is smart in the sense that it understands 

# Making data tidy

So far we have been lucky, our data is already in the "tidy format".

There are three main rules for a data being tidy

1. Each variable must have its own column.
2. Each observation must have its own row.
3. Each value must have its own cell.

Let's take for instance this data set. It's in a nice, Excel-y format and nicely human readable and you might consider writing something like this in excel to track things.

Tidy data is in fact another name [the third normal form](https://en.wikipedia.org/wiki/Third_normal_form) of database normalization. If you are not a computer scientist ignore this fact. If you are, feel free to vomit rainbows.

In [ ]:
table4a

What's wrong with this data? Part of the data is in the column headers! The variable is in the body of the table, but part of the identifier is in the column header.

To mutate this we `gather()` the data.


In [ ]:
tidy4 <- gather(table4a, `1999`, `2000`, key = "year", value = "cases")
tidy4

Now we can easily make aggregates on the data because it is in a tidy format.

E.g. the yearly means come simply enough.

In [ ]:
group_by(tidy4, country) %>%
    summarise(mean=mean(cases))

The inverse is also possible. Here there are two variables in the `rate` column.


In [ ]:
table2

To combat this we will need to `spread()` the values so that we have a single (Country, year) observation that contains two values, the cases and the count.

In [ ]:
spread(table2, key=type, value=count)

Now again the data is tidy. Each observation contains four variables
* country
* year
* cases
* population

It may not be intuitive that year and country are also variables but they are.

### Separating and uniting

Sometimes we have multiple variables in a same column, e.g.


In [ ]:
table3

In [ ]:
To combat this we have `separate()`. Let's see what this does.

In [ ]:
?separate

In [ ]:
# So this becomes
separate(table3, col=rate, into=c("cases", "population")

### Exercise

There is also an inverse of separate, called `unite()`. In the following case the century and year have been artificially split.

Use `unite()` to combine century and year to a canonical year and then separate the rate into two variables as previously to make this data tidy.

In [ ]:
table5

In [ ]:
tidy5 <- unite(table5, "year", sep="", century, year) %>%
    separate(col=rate, into=c("cases", "population"))


**Extra** what variable type is the "year" variable now? How could you `mutate` it to a number?

## Extra: Relational data

If you were excited about the third normal form earlier then you probably know about databases and relational algebra. If you don't know anything about databases this might be a bit advanced for you.

Tidyverse has tools for doing relational algebra in a simple way. 

In [ ]:
# load the dataset
library(nycflights13)

In [ ]:
head(planes)

In [ ]:
head(flights)

In [ ]:
head(weather)

The data is in a format where it is not needlessly duplicated. E.g. the information about an airplane is stored in a different table.

Let's make an **inner join**. Other sorts of joins are also available if required.


In [ ]:
flights %>%
    inner_join(planes, by="tailnum")

Okay, so now we can do all sorts of powerful aggregation.

In [ ]:
flights %>%
    inner_join(planes, by="tailnum") %>%
    group_by(manufacturer) %>%
    summarize(count=n()) %>%
    arrange(desc(count))

Can you feel the power at your fingertips?